In [1]:
from flask import Flask, request, jsonify, render_template
import os, tempfile, pickle, re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

app = Flask(__name__)

# --- Załaduj model i wagi ---
with open("meta_clf.pkl","rb") as f:
    meta_clf = pickle.load(f)
df_wagi = pd.read_pickle("df_wagi_per_kat.pkl")

# Definicje serwisów jak wcześniej
SITES = [
    {
        "name": "AI Image Detector",
        "url": "https://aiimagedetector.org/",
        "file_input_selector": "input[type='file']",
        "result_selector": "h4.text-lg span.text-base",
        "analyze_button_selector": "//button[contains(text(),'Analyze')]",
        "invert": True 
    },
    {
        "name": "The Detector AI",
        "url": "https://thedetector.ai/",
        "file_input_selector": "input[type='file']",
        "result_selector": "p.text-gray-400",
        "analyze_button_selector": "//button[contains(text(),'Analyze Image')]",
        "invert": False
    },
    {
        "name": "AI Detect Content",
        "url": "https://aidetectcontent.com/ai-generated-image-detector/",
        "file_input_selector": "input[type='file']",
        "result_section_id": "result-section",
        "fake_percentage_id": "fake_percentage",
        "analyze_button_selector": "//button[contains(text(),'Check Image')]",
        "invert": False
    }
]

modele = {
    'ai_image_detector': 'AI Image Detector',
    'the_detector_ai':   'The Detector AI',
    'ai_detect_content': 'Ai Detect Content'
}

def analyze_image_on_site(driver, site, img_path):
    driver.get(site["url"])
    wait = WebDriverWait(driver, 30)
    inp = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, site["file_input_selector"])))
    inp.send_keys(img_path)
    try:
        btn = wait.until(EC.element_to_be_clickable((By.XPATH, site["analyze_button_selector"])))
        btn.click()
    except:
        inp.send_keys(Keys.ENTER)
      # różne sposoby odczytu wyników:
    if site["name"] == "AI Detect Content":
        # czekamy na sekcję z ID
        wait.until(EC.visibility_of_element_located(
            (By.ID, site["result_section_id"])
        ))
        txt = driver.find_element(By.ID, site["fake_percentage_id"]).text
    elif site["name"] == "The Detector AI":
        # p.text-gray-400, szukamy % w kilku próbach
        txt = ""
        for _ in range(3):
            elems = driver.find_elements(By.CSS_SELECTOR, site["result_selector"])
            txt = next((e.text for e in elems if "%" in e.text), "")
            if txt:
                break
            time.sleep(5)
        if not txt:
            txt = driver.execute_script("""
                return [...document.querySelectorAll('p.text-gray-400')]
                  .find(el=>el.innerText.includes('%'))?.innerText||""
            """)
    else:  # AI Image Detector
        elem = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, site["result_selector"])
        ))
        txt = elem.text.strip()

    # parsowanie liczby
    m = re.search(r'(\d+(?:\.\d+)?)', txt)
    if not m:
        return None
    val = float(m.group(1))
    if site.get("invert", False):
        val = 100 - val
    return val / 100.0

def get_category_from_user(cat_str):
    if cat_str.lower() in ("building","animal","food"):
        return cat_str.lower()
    raise ValueError("Nieobsługiwana kategoria")

def load_selenium():
    opts = webdriver.EdgeOptions()
    opts.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Edge(options=opts)

@app.route("/", methods=["GET","POST"])
def index():
    if request.method=="POST":
        # 1) zapisz upload do tmp
        f = request.files["image"]
        cat = get_category_from_user(request.form["category"])
        _, ext = os.path.splitext(f.filename)
        tmp = tempfile.NamedTemporaryFile(suffix=ext, delete=False)
        f.save(tmp.name)

        # 2) zbieramy 3 prawdopodobieństwa
        driver = load_selenium()
        probs = {}
        for site in SITES:
            p = analyze_image_on_site(driver, site, tmp.name)
            # przypisz do klucza kolumny dataframe
            key = [k for k,v in modele.items() if v==site["name"]][0]
            probs[key] = p
        driver.quit()
        os.unlink(tmp.name)

        # 3) K3 stacking
        X = pd.DataFrame([probs])
        pred_k3 = meta_clf.predict(X)[0]

        # 4) zwróć wynik
        return render_template("result.html",
            decision = "AI-generated" if pred_k3 else "Real",
            probs=probs
        )
    return render_template("index.html")

if __name__=="__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\zonea\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
